In [1]:
import pandas as pd
import numpy as np

In [96]:
ids_df = pd.read_csv("./IDS_mapping.csv")
df = pd.read_csv("./diabetic_data.csv")

In [82]:
#splitting data of ids file
admission_type_id_df = ids_df.iloc[:8]

discharge_disposition_id_df = ids_df.iloc[10:40]
discharge_disposition_id_df = discharge_disposition_id_df.rename(columns={'admission_type_id': 'discharge_disposition_id'})


admission_source_id_df = ids_df.iloc[42:]
admission_source_id_df = admission_source_id_df.rename(columns={'admission_type_id': 'admission_source_id'})


In [88]:
discharge_disposition_id_df

,discharge_disposition_id,description
10,1,Discharged to home
11,2,Discharged/transferred to another short term h...
12,3,Discharged/transferred to SNF
13,4,Discharged/transferred to ICF
14,5,Discharged/transferred to another type of inpa...
15,6,Discharged/transferred to home with home healt...
16,7,Left AMA
17,8,Discharged/transferred to home under care of H...
18,9,Admitted as an inpatient to this hospital
19,10,Neonate discharged to another hospital for neo...


In [27]:
df.isnull().sum()

encounter_id                    0
patient_nbr                     0
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

### we have,

1. 30248 repeated patients
2. all unique encounter_id
3. 2273 patients with unknown race marked by '?'
4. 3 records with 'Unknown/Invalid' gender
5. all records of age
6. 98569 unknown weights marked by '?'
7. 9 admission_type_id
8. discharge_disposition_id {21, 26} missing
9. admission_source_id {12, 15, 16, 18, 19, 21, 23, 24} missing
10. Integer: number of days between admission and discharge
11. 40256 unknown payer_code marked by '?' with 17 distinct providers #why dataset info has 23 distinct values??
12. 49949 unknown medical_specialty marked by '?' with 72 distinct specialist  #why dataset info has 84 distinct values??
13. Number of lab tests performed during the encounter with min 1 and max 132
14. Number of procedures (other than lab tests) performed during the encounter min 0 and max 6
15. Number of distinct generic names administered during the encounter min 1 and max 81
16. Number of outpatient visits of the patient in the year preceding the encounter(vistis without hospitalizing).
17. Number of emergency visits of the patient in the year preceding the encounter
18. The number of times a patient is admitted to a hospital for medical care
19. The primary diagnosis (coded as first three digits of ICD9); 717 distinct values #why dataset info has 848 distinct values??
20. Secondary diagnosis (coded as first three digits of ICD9); 749 distinct values #why dataset info has 923 distinct values??
21. Additional secondary diagnosis (coded as first three digits of ICD9); 790 distinct values #why dataset info has 954 distinct values??
22. Number of diagnoses entered to the system
23. Indicates the range of the result or if the test was not taken. Values: >200, >300, normal, and none if not measured
24. If the test was not taken. Values: >8 if the result was greater than 8%, >7 if the result was greater than 7% but less than 8%, normal if the result was less than 7%, and none if not measured.
25. Whether the drug was prescribed or there was a change in the dosage.
26. Whether the drug was prescribed or there was a change in the dosage.
27. Whether the drug was prescribed or there was a change in the dosage.
28. Whether the drug was prescribed or there was a change in the dosage.
29. Whether the drug was prescribed or there was a change in the dosage.
30. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
31. Whether the drug was prescribed or there was a change in the dosage.
32. Whether the drug was prescribed or there was a change in the dosage.
33. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
34. Whether the drug was prescribed or there was a change in the dosage.
35. Whether the drug was prescribed or there was a change in the dosage.
36. Whether the drug was prescribed or there was a change in the dosage.
37. Whether the drug was prescribed or there was a change in the dosage.
38. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
39. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 3 values
40. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 1 value i.e No
41. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 1 value i.e No
42. Whether the drug was prescribed or there was a change in the dosage.
43. Whether the drug was prescribed or there was a change in the dosage.
44. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
45. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
46. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
47. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
48. Indicates if there was a change in diabetic medications.
49. Indicates if there was any diabetic medication prescribed.
50. Days to inpatient readmission. {'<30', '>30', 'No'}

In [101]:
print(df[df['medical_specialty'] == '?'].shape)

# has '?' in 2273 columns

(49949, 50)


In [119]:
df['max_glu_serum'].unique()

array([nan, '>300', 'Norm', '>200'], dtype=object)

In [120]:
df['A1Cresult'].unique()

array([nan, '>7', '>8', 'Norm'], dtype=object)

In [117]:
num = 1
for i in df.columns:
    print(num," ", i,":")
    a = df[i].unique()
    if i not in ['max_glu_serum', 'A1Cresult']:
        a.sort()
        print(a, len(a))
    num += 1

1   encounter_id :
[    12522     15738     16680 ... 443854148 443857166 443867222] 101766
2   patient_nbr :
[      135       378       729 ... 189445127 189481478 189502619] 71518
3   race :
['?' 'AfricanAmerican' 'Asian' 'Caucasian' 'Hispanic' 'Other'] 6
4   gender :
['Female' 'Male' 'Unknown/Invalid'] 3
5   age :
['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)'] 10
6   weight :
['>200' '?' '[0-25)' '[100-125)' '[125-150)' '[150-175)' '[175-200)'
 '[25-50)' '[50-75)' '[75-100)'] 10
7   admission_type_id :
[1 2 3 4 5 6 7 8] 8
8   discharge_disposition_id :
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24 25
 27 28] 26
9   admission_source_id :
[ 1  2  3  4  5  6  7  8  9 10 11 13 14 17 20 22 25] 17
10   time_in_hospital :
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 14
11   payer_code :
['?' 'BC' 'CH' 'CM' 'CP' 'DM' 'FR' 'HM' 'MC' 'MD' 'MP' 'OG' 'OT' 'PO' 'SI'
 'SP' 'UN' 'WC'] 18
12   medical_specialty :
['?' 'A

In [57]:
df['admission_type_id'].unique().shape

(8,)

In [41]:
df.shape

(101766, 50)

In [42]:
101766 - 71518

30248